In [2]:
from align.spark.schemas import ARCHIVE_ORG_SCHEMA
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CC-BY-license').getOrCreate()
input_catalogue_path = "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/EXPANDED_LICENSES_FILTERED_ACCESS.jsonl.gz"
df = spark.read.format('json').schema(ARCHIVE_ORG_SCHEMA).load(input_catalogue_path)
print(df.count())

63627


In [3]:
columns = [df.metadata.licenseurl, df.metadata.creator, df.metadata.title, df.metadata.credits]
licenses = ['https://creativecommons.org/licenses/by/4.0/',
'https://creativecommons.org/licenses/by/3.0/',
'http://creativecommons.org/licenses/by/2.0/',
'http://creativecommons.org/licenses/by/3.0/',
'https://creativecommons.org/licenses/by/3.0',
'http://creativecommons.org/licenses/by/2.5/',
'http://creativecommons.org/licenses/by/2.5/pl/',
'http://creativecommons.org/licenses/by/2.5/it/',
'http://creativecommons.org/licenses/by/4.0/',
'http://creativecommons.org/licenses/by/3.0/us/',
'http://creativecommons.org/licenses/by/2.5/ar/']
df = df.select(columns)
df.show()

+--------------------+--------------------+--------------------+--------------------+
| metadata.licenseurl|    metadata.creator|      metadata.title|    metadata.credits|
+--------------------+--------------------+--------------------+--------------------+
|http://creativeco...|Lamont-Clemens, Inc.|Tomorrow Always C...|Produced by Burne...|
|http://creativeco...|                AT86|00 Bienvenido A L...|                null|
|http://creativeco...|          Matrixland|00 Bienvenido A L...|                null|
|http://creativeco...|                Open|Open Friday | Apr...|                null|
|http://creativeco...|                Open|02777 A Open Apri...|                null|
|http://creativeco...|  Francisco Betances|Open Monday | Jun...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|                Open|Open Wednes

In [4]:
c = '`metadata.licenseurl`'
df.agg(F.count(F.when(F.isnull(c), c)).alias('NULL_Count')).show()

+----------+
|NULL_Count|
+----------+
|     48411|
+----------+



### Look NA values

In [4]:
df_pd = df.toPandas()
df_pd.head()

,metadata.licenseurl,metadata.creator,metadata.title,metadata.credits
0,http://creativecommons.org/publicdomain/mark/1.0/,"Lamont-Clemens, Inc.",Tomorrow Always Comes,Produced by Burnet Lamont; Directed by William...
1,http://creativecommons.org/publicdomain/mark/1.0/,AT86,00 Bienvenido A La Revolución De La Impresión ...,None
2,http://creativecommons.org/publicdomain/mark/1.0/,Matrixland,00 Bienvenido A La Revolución De La Impresión ...,None
3,http://creativecommons.org/publicdomain/mark/1.0/,Open,"Open Friday | Apr. 12, 2013",None
4,http://creativecommons.org/publicdomain/mark/1.0/,Open,02777 A Open April 17 13,None


In [5]:
df_pd.shape

(63627, 4)

In [6]:
df_pd.isna().sum()

metadata.licenseurl    48411
metadata.creator       49509
metadata.title         48411
metadata.credits       61617
dtype: int64

In [7]:
df_pd['metadata.licenseurl'].value_counts(normalize=True) * 100

http://creativecommons.org/publicdomain/mark/1.0/                                                                                                                                                27.563091
https://creativecommons.org/licenses/by/3.0/                                                                                                                                                     25.558623
http://creativecommons.org/licenses/publicdomain/                                                                                                                                                23.869611
http://creativecommons.org/publicdomain/zero/1.0/                                                                                                                                                11.041009
http://creativecommons.org/licenses/by/3.0/                                                                                                                                                 

In [8]:
df_pd[df_pd['metadata.licenseurl'].isna()]

,metadata.licenseurl,metadata.creator,metadata.title,metadata.credits
10,None,None,None,None
134,None,None,None,None
199,None,None,None,None
200,None,None,None,None
249,None,None,None,None
...,...,...,...,...
63589,None,None,None,None
63590,None,None,None,None
63591,None,None,None,None
63592,None,None,None,None


In [9]:
df_pd['metadata.licenseurl'].unique()

array(['http://creativecommons.org/publicdomain/mark/1.0/', None,
       'https://creativecommons.org/licenses/by/3.0/',
       'http://creativecommons.org/publicdomain/zero/1.0/',
       'http://creativecommons.org/licenses/by/3.0/',
       'https://creativecommons.org/licenses/by/3.0',
       'http://creativecommons.org/licenses/by/4.0/',
       'http://creativecommons.org/licenses/publicdomain/',
       'https://creativecommons.org/licenses/cc0/3.0/',
       'https://creativecommons.org/licenses/by/4.0/',
       'http://creativecommons.org/licenses/by/3.0/us/',
       'http://creativecommons.org/licenses/by/2.5/',
       'http://creativecommons.org/licenses/by/2.0/',
       'http://creativecommons.org/licenses/by/2.5/pl/',
       'http://creativecommons.org/licenses/by/2.5/it/',
       '["https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/"]',
       

## Filter values by  license

In [10]:
df = (df.withColumnRenamed('metadata.licenseurl','licenseurl').withColumnRenamed('metadata.creator', 'creator')
     .withColumnRenamed('metadata.title', 'title').withColumnRenamed('metadata.credits', 'credits'))
df = df.dropna(subset=['licenseurl'])
df.show()

+--------------------+--------------------+--------------------+--------------------+
|          licenseurl|             creator|               title|             credits|
+--------------------+--------------------+--------------------+--------------------+
|http://creativeco...|Lamont-Clemens, Inc.|Tomorrow Always C...|Produced by Burne...|
|http://creativeco...|                AT86|00 Bienvenido A L...|                null|
|http://creativeco...|          Matrixland|00 Bienvenido A L...|                null|
|http://creativeco...|                Open|Open Friday | Apr...|                null|
|http://creativeco...|                Open|02777 A Open Apri...|                null|
|http://creativeco...|  Francisco Betances|Open Monday | Jun...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|                Open|Open Wednes

In [11]:
regexp = r'\b(http|https)://creativecommons.org/licenses/by\b'
df = df.filter(df['licenseurl'].rlike(regexp))
df.show()

+--------------------+--------------------+--------------------+-------+
|          licenseurl|             creator|               title|credits|
+--------------------+--------------------+--------------------+-------+
|https://creativec...|     City of Holland|1-12-16 Early Bir...|   null|
|https://creativec...|     City of Holland|1-12-16 Planning ...|   null|
|https://creativec...|     City of Holland|1-18-17 City Council|   null|
|https://creativec...|     City of Holland|1-18-18 City Coun...|   null|
|https://creativec...|     City of Holland|1-21 City Council...|   null|
|https://creativec...|     City of Holland|1-22 City Council...|   null|
|https://creativec...|     City of Holland|1-23 Board of App...|   null|
|https://creativec...|     City of Holland|1-25-18 Board of ...|   null|
|https://creativec...|     City of Holland|1-26-17 Board of ...|   null|
|https://creativec...|     City of Holland|1-28-16 Board of ...|   null|
|https://creativec...|     City of Holland|1-3-18 C

## Types of licences

In [12]:
licenses_url = df.select('licenseurl').distinct().collect()
licenses_urls = []
for license in licenses_url:
    licenses_urls.append(license[0])
licenses_urls

['https://creativecommons.org/licenses/by/4.0/',
 '["https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/"]',
 'https://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/2.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'https://creativecommons.org/licenses/by/3.0',
 'http://creativecommons.org/licenses/by/2.5/',
 'http://creativecommons.org/licenses/by/2.5/pl/',
 'http://creativecommons.org/licenses/by/2.5/it/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/licenses/by/3.0/us/',
 'http://creativecommons.org/licenses/by/2.5/ar/']

## Convert to SQL

In [12]:
df.createOrReplaceTempView("CC_BY_creators")

In [13]:
spark.sql('SELECT * FROM CC_BY_creators').show()

+--------------------+--------------------+--------------------+-------+
|          licenseurl|             creator|               title|credits|
+--------------------+--------------------+--------------------+-------+
|https://creativec...|     City of Holland|1-12-16 Early Bir...|   null|
|https://creativec...|     City of Holland|1-12-16 Planning ...|   null|
|https://creativec...|     City of Holland|1-18-17 City Council|   null|
|https://creativec...|     City of Holland|1-18-18 City Coun...|   null|
|https://creativec...|     City of Holland|1-21 City Council...|   null|
|https://creativec...|     City of Holland|1-22 City Council...|   null|
|https://creativec...|     City of Holland|1-23 Board of App...|   null|
|https://creativec...|     City of Holland|1-25-18 Board of ...|   null|
|https://creativec...|     City of Holland|1-26-17 Board of ...|   null|
|https://creativec...|     City of Holland|1-28-16 Board of ...|   null|
|https://creativec...|     City of Holland|1-3-18 C

MapPartitionsRDD[43] at toJavaRDD at NativeMethodAccessorImpl.java:0

## Save as txt

In [ ]:
def myConcat(*cols):
    concat_columns = []
    for c in cols[:-1]:
        concat_columns.append(F.coalesce(c, F.lit("*")))
        concat_columns.append(F.lit(" "))  
    concat_columns.append(F.coalesce(cols[-1], F.lit("*")))
    return F.concat(*concat_columns)

df = df.withColumn("credits", myConcat(*df.columns)).select("credits")
df.coalesce(1).write.format("text").option("header", "false").mode("append").save("credits.txt")